In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import model_selection
import matplotlib.pyplot as plt
import torch
import scipy.stats as stats
import torch.nn as nn

In [2]:
# Lets grab our data from the pca transformation
X_numpy = np.load('pca_transformed_data_v1.npy', allow_pickle=True)
y_numpy = np.load('label.npy', allow_pickle=True)

In [3]:
X = torch.from_numpy(X_numpy.astype(np.float32)).squeeze()
y = torch.from_numpy(y_numpy.astype(np.float32))

In [4]:
#lets split our data
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2,
                                                                           random_state= 51)

In [5]:
#send the data into gpu
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
X_train,X_test, y_train,y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

In [6]:
# lets create a model for hyperparameter tuning
class NeuralNetworkWithHyper(nn.Module):
    def __init__(self,hidden_layers, num_hidden_units, input_size=18):
        super().__init__()
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.num_hidden_units = num_hidden_units
        
        # Create a list to hold the layers
        self.layers = nn.ModuleList()
        
        #add the input layer
        self.layers.append(nn.Linear(input_size, num_hidden_units))
        self.layers.append(nn.ReLU())
       
        #add hidden layers
        for i in range(hidden_layers - 1):
            self.layers.append(nn.Linear(num_hidden_units, num_hidden_units))
            self.layers.append(nn.ReLU())
        #add the output layer
        self.layers.append(nn.Linear(num_hidden_units, 1))
        
        #add forward function
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

### Study #1

In [8]:
#Lets set our loss function and optimizer
import optuna

def objective(trial):
    
    #define the search space
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3)
    hidden_layers = trial.suggest_int("hidden_layers", 1, 1)
    num_hidden_units = trial.suggest_int("num_hidden_units", 26, 46)
    epoch_count = trial.suggest_int("epoch_count", 9000, 12000)
    weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-3)
    
    #create the model
    model = NeuralNetworkWithHyper(hidden_layers, 
                                   num_hidden_units, 
                                   random_state=42).to('mps')
    optimizer = torch.optim.SGD(model.parameters(), learning_rate, weight_decay)
    loss_fn = torch.nn.L1Loss()
    
    #testing loop
    for epoch in range(epoch_count):
        #convert into train
        model.train()

        #fit data
        train_pred = model(X_train)

        #calculate loss
        train_loss = loss_fn(train_pred, y_train)


        #zero gradients
        optimizer.zero_grad()

        #back propagation
        train_loss.backward()

        #gradient descent
        optimizer.step()

        #testing
        model.eval()
        with torch.inference_mode():
            test_pred = model(X_test)
            test_loss = loss_fn(test_pred, y_test)
            #spear_pred = test_pred.to('cpu')
            
    #spearman correlation coefficient       
    validation_metric = stats.spearmanr(y_test.to('cpu'), spear_pred).statistic
    return validation_metric

#Create an Optuna Study
study = optuna.create_study(direction='minimize')

#Perform hyperparameter optimization
study.optimize(objective, n_trials=100)

#Print best spearman correlation coefficient
best_validation_metric = study.best_value

[I 2023-11-08 01:10:59,534] A new study created in memory with name: no-name-4b741efb-6f3d-4864-baaa-c1f611749999
[I 2023-11-08 01:11:24,026] Trial 0 finished with value: 0.24406498283896852 and parameters: {'learning_rate': 0.0005128259391461643, 'hidden_layers': 1, 'num_hidden_units': 44, 'epoch_count': 9792, 'weight_decay': 0.0005870800923048699}. Best is trial 0 with value: 0.24406498283896852.
[I 2023-11-08 01:11:50,508] Trial 1 finished with value: 0.24609861452807952 and parameters: {'learning_rate': 0.0006609166871203145, 'hidden_layers': 1, 'num_hidden_units': 39, 'epoch_count': 10827, 'weight_decay': 0.000990002449065781}. Best is trial 1 with value: 0.24609861452807952.
[I 2023-11-08 01:12:15,133] Trial 2 finished with value: 0.19677047615824278 and parameters: {'learning_rate': 0.0006944468566519496, 'hidden_layers': 1, 'num_hidden_units': 40, 'epoch_count': 9883, 'weight_decay': 0.0006828474675480177}. Best is trial 1 with value: 0.24609861452807952.
[I 2023-11-08 01:12:36

In [9]:
 best_trial = study.best_trial
score = best_trial.value
best_params = best_trial.params
print(f'Best score: {score} \nBest Hyperparameters: ')
for key,value in best_params.items():
    print(f'{key}: {value}')

Best score: 0.30299451144078343 
Best Hyperparameters: 
learning_rate: 0.0009356333110500949
hidden_layers: 1
num_hidden_units: 38
epoch_count: 9737
weight_decay: 0.0009295932677289612


It seems like our model is responsing well to the transformation, centering + scaling, and PCA that we have done on our data. We will now narrow the range of our parameters in response to the hyperparameters of our first study. Another thing that I want to explore is how the model reacts to removing the Relu function.

### Study #2

In [10]:
def objective(trial):
    #define the search space
    learning_rate= trial.suggest_float("learning_rate", 1e-4, 1e-3)
    hidden_layers= trial.suggest_int("hidden_layers", 1, 1)
    num_hidden_units= trial.suggest_int("num_hidden_units", 26, 46)
    epoch_count= trial.suggest_int("epoch_count", 9000, 11000)
    weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-3)
    
    #create the model
    model = NeuralNetworkWithHyper(hidden_layers, 
                                   num_hidden_units,
                                  random_state=52).to('mps')
    optimizer = torch.optim.SGD(model.parameters(), learning_rate, weight_decay)
    loss_fn = torch.nn.L1Loss()
    
    #testing loop
    for epoch in range(epoch_count):
        #convert into train
        model.train()

        #fit data
        train_pred = model(X_train)

        #calculate loss
        train_loss = loss_fn(train_pred, y_train)


        #zero gradients
        optimizer.zero_grad()

        #back propagation
        train_loss.backward()

        #gradient descent
        optimizer.step()

        #testing
        model.eval()
        with torch.inference_mode():
            test_pred = model(X_test)
            test_loss = loss_fn(test_pred, y_test)
            spear_pred = test_pred.to('cpu')
            
    #spearman correlation evaluation        
    validation_metric = stats.spearmanr(y_test.to('cpu'), spear_pred).statistic
    return validation_metric

#Create an Optuna Study
study = optuna.create_study(direction='maximize')

#Perform hyperparameter optimization
study.optimize(objective, n_trials=100)

#Print best spearman correlation coefficient
best_validation_metric = study.best_value

[I 2023-11-08 06:15:33,110] A new study created in memory with name: no-name-a9b4e26f-a76b-45e6-9065-26371d5bf86c
[I 2023-11-08 06:16:00,235] Trial 0 finished with value: 0.24148545796355314 and parameters: {'learning_rate': 0.0006974137941492071, 'hidden_layers': 1, 'num_hidden_units': 31, 'epoch_count': 10957, 'weight_decay': 0.0008270561192446379}. Best is trial 0 with value: 0.24148545796355314.
[I 2023-11-08 06:16:23,390] Trial 1 finished with value: 0.15374210677147332 and parameters: {'learning_rate': 0.0002190059353282515, 'hidden_layers': 1, 'num_hidden_units': 26, 'epoch_count': 9746, 'weight_decay': 0.00048609200511697735}. Best is trial 0 with value: 0.24148545796355314.
[I 2023-11-08 06:16:48,764] Trial 2 finished with value: 0.16996088064878057 and parameters: {'learning_rate': 0.0002942479004110647, 'hidden_layers': 1, 'num_hidden_units': 39, 'epoch_count': 10726, 'weight_decay': 0.0002514674002580159}. Best is trial 0 with value: 0.24148545796355314.
[I 2023-11-08 06:17

In [11]:
best_trial = study.best_trial
best_params = best_trial.params
best_score = best_trial.value
print(f'Best Score: {best_score} \nHyperparameters: ')
for key,value in best_params.items():
    print(f'{key}: {value}')

Best Score: 0.3176745937917269 
Hyperparameters: 
learning_rate: 0.0009289305355089368
hidden_layers: 1
num_hidden_units: 34
epoch_count: 9308
weight_decay: 0.0007777563951641034


### Study #3 Random Forest

In [24]:
X_train, X_test, y_train, y_test = X_train.to('cpu'), X_test.to('cpu'), y_train.to('cpu').squeeze(), y_test.to('cpu').squeeze()

In [25]:
X_train.numpy(), X_test.numpy(), y_train.numpy(), y_test.numpy()

(array([[-1.8840996 , -1.2298406 , -0.08687859, ...,  0.21039778,
         -0.24490474, -1.0782467 ],
        [-3.0310247 , -1.0652962 , -0.6179891 , ...,  0.06913669,
         -0.13375966, -0.15979725],
        [-2.4773939 , -0.24320649, -2.6922863 , ...,  0.37434563,
         -0.5727486 ,  0.52930886],
        ...,
        [ 4.008305  , -3.0478313 , -0.4981839 , ..., -0.25590926,
         -0.03921561,  0.6463667 ],
        [-2.3074343 ,  2.9922462 ,  2.0321922 , ..., -0.24898356,
          0.5775757 ,  0.80868113],
        [ 3.7405992 , -0.9258261 ,  0.52959573, ..., -0.14529   ,
         -0.31167406,  0.21507934]], dtype=float32),
 array([[-2.7682261 , -0.8817636 ,  2.15579   , ...,  0.8664899 ,
          0.33826873,  0.36335376],
        [ 1.9744093 ,  5.584851  , -1.1742501 , ..., -1.0187863 ,
         -0.7615747 ,  0.53597295],
        [ 1.2637563 , -2.2719896 ,  0.30258185, ...,  1.1502167 ,
          0.83102214,  0.27453494],
        ...,
        [-0.25826934,  1.5601684 ,  1.5

In [26]:
y_train.shape, X_train.shape

(torch.Size([1195]), torch.Size([1195, 18]))

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    
    #define the search space
    n_estimators= trial.suggest_int("n_estimators", 100, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_float('min_samples_split', 0.1, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.1, 0.5)
    
    #create the model
    model = RandomForestRegressor(n_estimators,
                                 max_depth=max_depth,
                                 min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf,
                                 random_state=42)
    
    #training & testing loop
    model.fit(X_train, y_train)
    model_predictions = model.predict(X_test)
    
    #make predictions on data set
    mse = mean_squared_error(y_test, model_predictions)
    return mse


#Create an Optuna Study
study_3 = optuna.create_study(direction='minimize')

#Perform hyperparameter optimization
study_3.optimize(objective, n_trials=100)

#Print best spearman correlation coefficient
best_validation_metric = study_3.best_value

[I 2023-11-08 07:10:18,089] A new study created in memory with name: no-name-c96795f6-48c6-4108-9300-6d195490255d
[I 2023-11-08 07:10:18,461] Trial 0 finished with value: 0.766444713967228 and parameters: {'n_estimators': 876, 'max_depth': 7, 'min_samples_split': 0.7474505613834166, 'min_samples_leaf': 0.27800876599230734}. Best is trial 0 with value: 0.766444713967228.
[I 2023-11-08 07:10:18,674] Trial 1 finished with value: 0.7663947576396374 and parameters: {'n_estimators': 539, 'max_depth': 30, 'min_samples_split': 0.16564730785898213, 'min_samples_leaf': 0.3284254470655301}. Best is trial 1 with value: 0.7663947576396374.
[I 2023-11-08 07:10:18,968] Trial 2 finished with value: 0.7413064114503799 and parameters: {'n_estimators': 211, 'max_depth': 14, 'min_samples_split': 0.465179543553217, 'min_samples_leaf': 0.1605275779823189}. Best is trial 2 with value: 0.7413064114503799.
[I 2023-11-08 07:10:19,095] Trial 3 finished with value: 0.7664167687107659 and parameters: {'n_estimator

In [31]:
best_trial = study_3.best_trial
params = best_trial.params
score = best_trial.value
print(f'Best score: {score}\nHyperparameters: ')
for key,value in params.items():
    print(f'{key}: {value}')

Best score: 0.7214074424806814
Hyperparameters: 
n_estimators: 364
max_depth: 28
min_samples_split: 0.13216427917075566
min_samples_leaf: 0.1002395644210297


### Neural Network
Best Score: 0.3176745937917269 
* learning_rate: 0.0009289305355089368
* hidden_layers: 1
* num_hidden_units: 34
* epoch_count: 9308
* weight_decay: 0.0007777563951641034